
<div dir=rtl >
<h1>قوانین انجمنی</h1>

قوانین انجمنی روابط و وابستگی‌های متقابل بین مجموعه بزرگی از اقلام داده‌ای را نشان می‌دهند. 

مثال متداول در رابطه با کشف قوانین انجمنی "تحلیل سبد خرید" است.  در این فرایند با توجه به اقلام مختلفی که مشتریان در سبد خریدشان قرار می‌دهند، عادات و رفتار خرید مشتریان مورد تحلیل قرار می‌گیرد و می‌توان با شناسایی ارتباط بین محصولات، الگوهای تکرار شونده در حین خرید را بدست آورد.

سه پارامتر مهم:
- Support  میزان محبوبیت یک مجموعه آیتم را با توجه به دفعات حضور در تراکنش‌ها نشان می‌دهد.
- Confidence  احتمال خرید آیتم y در صورت خرید آیتم x را نشان می‌دهد. x -> y
- Lift   ترکیب دو پارامتر بالا می‌باشد.

برای پیاده سازی قوانین انجمنی در این تمرین، از الگوریتم Apriori که یکی از محبوب‌ترین و کارآمدترین الگوریتم‌های این حوزه است استفاده می‌کنیم.


<font color='Green'> پرسش:  تاثیر مقادیر مختلف پارامتر Lift را در احتمال وقوع نتیجه بررسی کنید.  </font>


</div>




<div dir=rtl >
<font color='Blue'>
می‌دانیم اگر رخداد دو آیتم‌ست مستقل از هم باشند رابطه‌ی زیر برای آن‌ها برقرار است:
<br>
P(A∪B)=P(A)P(B)
<br>
و در صورت برقرار نبودن این رابطه آیتم‌ست‌ها وابسته یا correlated هستند.
متغیر لیفت نیز بر همین مبنا به صورت کلی به شکل زیر تعریف می‌شود:
<br>
lift(A,B) = P(A∪B)/(P(A)P(B))
<br>
و در association ruleها به شکل زیر درمی‌آید:
<br>
A =>B
<br>
lift = confidence(A=>B)/support(B)
<br>
که با توجه به چیزی که گفته شد اگر این مقدار برابر با یک باشد یعنی آیتم‌ست‌ها مستقل از هم واقع می‌شوند. اگر بزرگ‌تر از 
یک باشد به این معناست که positively correlated هستند یعنی وقوع یکی (مثلا A) احتمال وقوع دیگری را قوی‌تر می‌کند.
<br>
نهایتا اگر مقدار لیفت کوچک‌تر از یک باشد به این معناست که آیتم‌ست‌ها negatively correlated هستند یعنی حضور یک آیتم‌ست احتمالا باعث واقع نشدن آیتم‌ست دیگر (B) می‌شود.
    </font>
</div>


<div dir=rtl >
<h1>الگوریتم Apriori</h1>

روش کار الگوریتم به این صورت است که یک  حداقل مقدار پشتیبانی در نظر گرفته    می‌شود و تکرارها با مجموعه آیتم‌های مکرر اتفاق می‌افتند. در صورتیکه  مجموعه‌ها و زیرمجموعه‌ها مقدار پشتیبانی کمتر از حد آستانه داشته باشند حذف می‌شوند. این روند تا زمانیکه امکان حذف وجود نداشته باشد ادامه می‌یابد. 

در این بخش از تمرین می‌خواهیم الگوریتم apriori را بر روی مجموعه داده Hypermarket_dataset که شامل سفارشات خرید افراد از فروشگاه های مواد غذایی است اعمال کنیم.


</div>





<div dir=rtl >
<h1>آماده‌سازی داده</h1>

<font color='Green'>پرسش: برای شروع کار باید داده‌های دیتاست را در قالب ماتریس پراکنده که محصولات  خریداری شده در ستون‌ و شماره سفارشات خرید به عنوان شاخص (index) هستند آماده کنید. 


برای راحتی کار محصولات خریداری شده در هر سفارش را با اعداد 0,1 کدگذاری کنید.

 نمونه ماتریس خروجی شما:

<img src="https://drive.google.com/uc?id=1eD0jan1ZbeYqSklgK--ks7oeY-MyTA3p"></img>

</div>




In [42]:
import numpy as np
import pandas as pd
!pip install mlxtend
import matplotlib.pyplot as plt
 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<div dir=rtl >
    <font color='Blue'>
ابتدا داده‌ها را از فایل csv می‌خوانیم و سپس روی آن‌ها iterate می‌کنیم و به ازای هر member یک دیکشنری از آیتم‌های خریداری شده تشکیل می‌دهیم و نهایتا این دیکشنری‌ها را به dataframe تبدیل می‌کنیم و حاصل را چاپ می‌کنیم که همانطور که مشخص است نتیجه نشان می‌دهد هر خریدار کدامیک از ۱۶۷ item را خریداری کرده است.
    </font>
</div>

In [43]:
hypermarket_data = pd.read_csv('Hypermarket_dataset.csv')
dicts = []
indices = []
for index, row in hypermarket_data.iterrows():
    if row['Member_number'] not in indices:
        indices.append(row['Member_number'])
        new_mem = {}
        new_mem[row['itemDescription']] = 1
        dicts.append(new_mem)
    else:
        dicts[indices.index(row['Member_number'])][row['itemDescription']] = 1



In [44]:
df = pd.DataFrame(dicts, index = indices) 

df = df.replace(np.nan, 0)
df

,tropical fruit,long life bakery product,meat,sugar,rolls/buns,semi-finished bread,whole milk,citrus fruit,candy,napkins,...,rubbing alcohol,cooking chocolate,potato products,bags,organic products,make up remover,baby cosmetics,toilet cleaner,kitchen utensil,preservation products
1808,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2552,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2300,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3037,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



<div dir=rtl >
<h1>شناسایی الگوهای پر تکرار</h1>

<font color='Green'> پرسش:  با اعمال الگوریتم apriori و به ازای مقدار min_support = 0.07 تمامی الگوهای پر تکرار را تولید نمایید. 

</div>




<div dir=rtl >
    <font color='Blue'>
    با استفاده از تابع apriori و تعیین کردن 0.07 به عنوان minsup می‌توانیم frequent itemsetها را به دست آوریم.
    </font>
</div>

In [45]:
frequent_itemsets = apriori(df, min_support=0.07, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.233710,(tropical fruit)
1,0.349666,(rolls/buns)
2,0.458184,(whole milk)
3,0.185480,(citrus fruit)
4,0.081324,(napkins)
...,...,...
78,0.075423,"(sausage, yogurt)"
79,0.097486,"(soda, yogurt)"
80,0.076193,"(bottled water, soda)"
81,0.082093,"(other vegetables, rolls/buns, whole milk)"



<div dir=rtl >
<h1>استخراج قوانین انجمنی</h1>


<font color='Green'> پرسش: تابعی بنویسید که با گرفتن دو ورودی confidence و lift قوانین انجمنی حاصل را در خروجی نشان دهد. </br>
برای دو حالت خروجی خود را در گزارش ثبت کنید.


</div>





<div dir=rtl >
    <font color='Blue'>
با استفاده از تابع association_rules و با دادن ورودی frequent_itemsets و اعمال thresholdهای ورودی برای confidence و lift قوانین مورد نظر را به دست می‌آوریم.    </font>
</div>

In [64]:
def generate_association_rules(frequent_itemsets, confidence, lift):
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence)
    return rules[(rules['lift'] >= lift)]
    

<div dir=rtl >
    <font color='Blue'>
در این بخش نمونه خروجی با مینیمم confidence برابر با ۵۰٪ و مینیمم lift برابر با ۱.۱۵ را مشاهده می‌کنیم.   </font>
</div>

In [80]:
from IPython.display import HTML
HTML(generate_association_rules(frequent_itemsets, 0.5,1.15).to_html(index=False))

antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
(shopping bags),(whole milk),0.168291,0.458184,0.091329,0.542683,1.184422,0.014220,1.184772
(domestic eggs),(whole milk),0.133145,0.458184,0.070292,0.527938,1.152242,0.009287,1.147766
(canned beer),(whole milk),0.165213,0.458184,0.087224,0.527950,1.152268,0.011526,1.147795
(bottled beer),(whole milk),0.158799,0.458184,0.085428,0.537964,1.174124,0.012669,1.172672
(yogurt),(whole milk),0.282966,0.458184,0.150590,0.532185,1.161510,0.020940,1.158185
"(other vegetables, rolls/buns)",(whole milk),0.146742,0.458184,0.082093,0.559441,1.220996,0.014859,1.229837
"(other vegetables, yogurt)",(whole milk),0.120318,0.458184,0.071832,0.597015,1.303003,0.016704,1.344507


<div dir=rtl >
    <font color='Blue'>
در این بخش نمونه خروجی با مینیمم confidence برابر با ۳۵٪ و مینیمم lift برابر با ۱.۲ را مشاهده می‌کنیم.   </font>
</div>

In [83]:
HTML(generate_association_rules(frequent_itemsets, 0.35,1.2).to_html(index=False))

antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
(sausage),(yogurt),0.206003,0.282966,0.075423,0.366127,1.293892,0.017132,1.131196
"(other vegetables, rolls/buns)",(whole milk),0.146742,0.458184,0.082093,0.559441,1.220996,0.014859,1.229837
"(other vegetables, whole milk)",(rolls/buns),0.191380,0.349666,0.082093,0.428954,1.226753,0.015174,1.138847
"(rolls/buns, whole milk)",(other vegetables),0.178553,0.376603,0.082093,0.459770,1.220834,0.014850,1.153947
"(other vegetables, whole milk)",(yogurt),0.191380,0.282966,0.071832,0.375335,1.326434,0.017678,1.147870
"(other vegetables, yogurt)",(whole milk),0.120318,0.458184,0.071832,0.597015,1.303003,0.016704,1.344507
"(yogurt, whole milk)",(other vegetables),0.150590,0.376603,0.071832,0.477002,1.266589,0.015119,1.191967
